In [ ]:
!pip install datasets librosa soundfile

In [ ]:
from datasets import load_dataset

dataset = load_dataset('dataset')
dataset

In [ ]:
print(dataset['train'][0])

Analyzing the volume levels in audio using RMS

In [ ]:
import librosa
import numpy as np

# Function to compute RMS for a given audio file
def compute_rms(audio_path):
    y, sr = librosa.load(audio_path)
    rms_values = librosa.feature.rms(y=y) # get rms values of this file
    mean_rms = np.mean(rms_values) # get mean of the rms values
    std_rms = np.std(rms_values) # get std of the rms values
    return mean_rms, std_rms

rms_values_list = []
mean_rms_list = []
std_rms_list = []

for i in range(len(dataset['train'])):
    audio_path = dataset['train'][i]['audio']['path'] 
    mean_rms, std_rms = compute_rms(audio_path)
    mean_rms_list.append(mean_rms)
    std_rms_list.append(std_rms)

overall_mean = np.mean(mean_rms_list) # get mean of the mean of each file
overall_std = np.std(mean_rms_list) # get std of the std of each file

print(f"Overall Mean of RMS values: {overall_mean}")
print(f"Overall Standard Deviation of RMS values: {overall_std}")

Since the std doesn't vary much from the mean, the volume level is consistent across the audios.

Plot the mean rms values for each file:

In [ ]:
import matplotlib.pyplot as plt

# Plot the mean RMS values across all audio files
plt.boxplot(mean_rms_list)
plt.title('Variation of Mean RMS Across Audio Files')
plt.ylabel('Mean RMS')
plt.show()

It shows outliers even though the std didn't differ. This could be because some audios are longer while majority are short, which can affect the RMS values.

In [ ]:
# import numpy as np

# example = minds[0]
# input_features = example["input_features"]

# plt.figure().set_figwidth(12)
# librosa.display.specshow(
#     np.asarray(input_features[0]),
#     x_axis="time",
#     y_axis="mel",
#     sr=feature_extractor.sampling_rate,
#     hop_length=feature_extractor.hop_length,
# )
# plt.colorbar()

In [ ]:
# Load the audio
y, sr = librosa.load('my_dataset_female/female_audio_segments_1/1_1.mp3')

# Compute the spectrogram
D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)

# Plot the spectrogram
plt.figure(figsize=(10, 6))
librosa.display.specshow(D, x_axis='time', y_axis='log', sr=sr)
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram of Audio')
plt.show()
